In [103]:
import os
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel


#NLTK-------------------------------
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer


# Import libraries for feature 
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from mlxtend.feature_selection import SequentialFeatureSelector as sfs


from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,classification_report
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")


[nltk_data] Downloading package punkt to /Users/rajatsuri/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [104]:
os.getcwd()


'/Users/rajatsuri/Desktop/Padhai/CIS 508/assig4'

In [105]:
#Read files

textData = pd.read_csv('Comments.csv') #creates a dataframe

CustInfoData = pd.read_csv('Customers.csv')  #creates a dataframe

print(textData.shape)
print(CustInfoData.shape)


(2070, 2)
(2070, 17)


In [106]:
#Extract target column from Customer Info file
y_train = CustInfoData["TARGET"]
X_train = CustInfoData.drop(columns=["TARGET"]) #extracting training data without the target column
                     
print(X_train.shape)
print(textData.shape)
print(textData.head())
print(y_train)

(2070, 16)
(2070, 2)
     ID                                           Comments
0  1309  Does not like the way the phone works. It is t...
1  3556  Wanted to know the nearest store location. Wan...
2  2230  Wants to know how to do text messaging. Referr...
3  2312  Asked how to disable call waiting. referred hi...
4  3327  Needs help learning how to use the phone. I su...
0       Cancelled
1         Current
2         Current
3         Current
4       Cancelled
          ...    
2065    Cancelled
2066    Cancelled
2067    Cancelled
2068    Cancelled
2069    Cancelled
Name: TARGET, Length: 2070, dtype: object


In [107]:
#Tokenize - Split the sentences to lists of words
textData['CommentsTokenized'] = textData['Comments'].apply(word_tokenize)

export_csv = textData.to_csv('/Users/rajatsuri/Desktop/Padhai/CIS 508/assig4/textData.csv')


In [108]:
# Use SnowballStemmer stemmer.
stemmer = SnowballStemmer("english")

#Now do stemming - create a new dataframe to store stemmed version
newTextDataSS=pd.DataFrame()
newTextDataSS=textData.drop(columns=["CommentsTokenized","Comments"])
newTextDataSS['CommentsTokenizedStemmed'] = textData['CommentsTokenized'].apply(lambda x: [stemmer.stem(y) for y in x]) # Stem every word.

export_csv = newTextDataSS.to_csv('/Users/rajatsuri/Desktop/Padhai/CIS 508/assig4/newTextDataSS.csv')


In [109]:
# Use PorterStemmer stemmer
stemmer = PorterStemmer ()

#Now do stemming - create a new dataframe to store stemmed version
newTextDataPS=pd.DataFrame()
newTextDataPS=textData.drop(columns=["CommentsTokenized","Comments"])
newTextDataPS['CommentsTokenizedStemmed'] = textData['CommentsTokenized'].apply(lambda x: [stemmer.stem(y) for y in x]) # Stem every word.

export_csv = newTextDataPS.to_csv('/Users/rajatsuri/Desktop/Padhai/CIS 508/assig4/newTextDataPS.csv')



In [110]:

#Join stemmed strings
newTextDataSS['CommentsTokenizedStemmed'] = newTextDataSS['CommentsTokenizedStemmed'].apply(lambda x: " ".join(x))

export_csv = newTextDataSS.to_csv ('/Users/rajatsuri/Desktop/Padhai/CIS 508/assig4/newTextDataSS.csv')

In [111]:
#Do Bag-Of-Words model - Term - Document Matrix
#Learn the vocabulary dictionary and return term-document matrix.
#count_vect = CountVectorizer(stop_words=None)
count_vect = CountVectorizer(stop_words='english',lowercase=False)
TD_counts = count_vect.fit_transform(newTextDataSS.CommentsTokenizedStemmed)
print(TD_counts.shape)
print(TD_counts.dtype)
print(count_vect.get_feature_names())
#print(TD_counts)
DF_TD_Counts=pd.DataFrame(TD_counts.toarray())
print(DF_TD_Counts)
export_csv = DF_TD_Counts.to_csv('/Users/rajatsuri/Desktop/Padhai/CIS 508/assig4/DF_TD_Counts.csv')


(2070, 354)
int64
['3399', '3g', 'abysm', 'access', 'accessori', 'adapt', 'add', 'addit', 'additon', 'address', 'adit', 'adress', 'advertis', 'afraid', 'alway', 'angel', 'angri', 'ani', 'anoth', 'anyth', 'anytim', 'area', 'asap', 'ask', 'bad', 'basic', 'bateri', 'batteri', 'becaus', 'believ', 'better', 'bigger', 'book', 'bought', 'brain', 'bring', 'built', 'busi', 'button', 'buy', 'cancel', 'cancer', 'car', 'care', 'carrier', 'caus', 'cc', 'cell', 'certain', 'chang', 'charg', 'charger', 'check', 'chip', 'citi', 'claim', 'cleariti', 'cold', 'comapr', 'compani', 'compar', 'competit', 'complain', 'complaint', 'concept', 'connect', 'consisit', 'consist', 'constan', 'contact', 'continu', 'contract', 'correct', 'cost', 'coupl', 'cover', 'coverag', 'creat', 'credit', 'cstmer', 'cstmr', 'current', 'cust', 'custom', 'customr', 'date', 'day', 'dead', 'decent', 'defect', 'deo', 'did', 'die', 'differ', 'difficult', 'digiti', 'direct', 'disabl', 'doe', 'don', 'dont', 'drop', 'dure', 'easier', 'effe

In [112]:
#Compute TF-IDF Matrix
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(TD_counts)
print(X_train_tfidf.shape)
DF_TF_IDF=pd.DataFrame(X_train_tfidf.toarray())
print(DF_TF_IDF)
export_csv= DF_TF_IDF.to_csv('/Users/rajatsuri/Desktop/Padhai/CIS 508/assig4/DF_TF_IDF.csv')


(2070, 354)
      0    1    2    3        4    5    6    7    8         9    ...  344  \
0     0.0  0.0  0.0  0.0  0.00000  0.0  0.0  0.0  0.0  0.000000  ...  0.0   
1     0.0  0.0  0.0  0.0  0.27568  0.0  0.0  0.0  0.0  0.000000  ...  0.0   
2     0.0  0.0  0.0  0.0  0.00000  0.0  0.0  0.0  0.0  0.000000  ...  0.0   
3     0.0  0.0  0.0  0.0  0.00000  0.0  0.0  0.0  0.0  0.000000  ...  0.0   
4     0.0  0.0  0.0  0.0  0.00000  0.0  0.0  0.0  0.0  0.000000  ...  0.0   
...   ...  ...  ...  ...      ...  ...  ...  ...  ...       ...  ...  ...   
2065  0.0  0.0  0.0  0.0  0.00000  0.0  0.0  0.0  0.0  0.000000  ...  0.0   
2066  0.0  0.0  0.0  0.0  0.00000  0.0  0.0  0.0  0.0  0.000000  ...  0.0   
2067  0.0  0.0  0.0  0.0  0.00000  0.0  0.0  0.0  0.0  0.000000  ...  0.0   
2068  0.0  0.0  0.0  0.0  0.00000  0.0  0.0  0.0  0.0  0.772949  ...  0.0   
2069  0.0  0.0  0.0  0.0  0.00000  0.0  0.0  0.0  0.0  0.000000  ...  0.0   

      345  346       347  348  349  350  351  352  353  
0     

In [113]:
#Merge files

print(CustInfoData.shape)
X_train = CustInfoData.drop(columns=["TARGET"]) #extracting training data without the target column
print(X_train.shape)
combined=pd.concat([X_train, DF_TF_IDF], axis=1)
print(combined.shape)
print(combined)
export_csv= combined.to_csv('/Users/rajatsuri/Desktop/Padhai/CIS 508/assig4/combined.csv')



(2070, 17)
(2070, 16)
(2070, 370)
        ID Sex Status  Children  Est_Income Car_Owner   Usage        Age  \
0        1   F      S         1    38000.00         N  229.64  24.393333   
1        6   M      M         2    29616.00         N   75.29  49.426667   
2        8   M      M         0    19732.80         N   47.25  50.673333   
3       11   M      S         2       96.33         N   59.01  56.473333   
4       14   F      M         2    52004.80         N   28.14  25.140000   
...    ...  ..    ...       ...         ...       ...     ...        ...   
2065  3821   F      S         0    78851.30         N   29.04  48.373333   
2066  3822   F      S         1    17540.70         Y   36.20  62.786667   
2067  3823   F      M         0    83891.90         Y   74.40  61.020000   
2068  3824   F      M         2    28220.80         N   38.95  38.766667   
2069  3825   F      S         0    28589.10         N  100.28  15.600000   

      RatePlan  LongDistance  ...  344  345  346     

In [114]:
#Do one Hot encoding for categorical features
X_cat = ["Sex","Status","Car_Owner","Paymethod","LocalBilltype","LongDistanceBilltype"]
#X_cat = combined.select_dtypes(exclude=['int','float64'])
print(X_cat)
combined_one_hot = pd.get_dummies(combined,columns=X_cat)
print(combined_one_hot.shape)
export_csv= combined_one_hot.to_csv('/Users/rajatsuri/Desktop/Padhai/CIS 508/assig4/combined_one_hot.csv')


['Sex', 'Status', 'Car_Owner', 'Paymethod', 'LocalBilltype', 'LongDistanceBilltype']
(2070, 378)


In [115]:
#Drop ID column as well before feature selection
combined_one_hot = combined_one_hot.drop(columns=["ID"])

In [116]:
#Feature selection using filtering
y_train = CustInfoData["TARGET"]
new_combined_one_hot_F = SelectKBest(score_func=chi2, k=100).fit_transform(combined_one_hot,y_train)
print(new_combined_one_hot_F.shape)

new_combined_one_hot_F= pd.DataFrame(new_combined_one_hot_F)
print(new_combined_one_hot_F)

export_csv= new_combined_one_hot_F.to_csv('/Users/rajatsuri/Desktop/Padhai/CIS 508/assig4/new_combined_one_hot_F.csv')


(2070, 100)
       0         1       2          3      4     5       6    7    8   \
0     1.0  38000.00  229.64  24.393333  23.56  0.00  206.08  0.0  0.0   
1     2.0  29616.00   75.29  49.426667  29.78  0.00   45.50  0.0  0.0   
2     0.0  19732.80   47.25  50.673333  24.81  0.00   22.44  0.0  0.0   
3     2.0     96.33   59.01  56.473333  26.13  0.00   32.88  1.0  0.0   
4     2.0  52004.80   28.14  25.140000   5.03  0.00   23.11  0.0  0.0   
...   ...       ...     ...        ...    ...   ...     ...  ...  ...   
2065  0.0  78851.30   29.04  48.373333   0.37  0.00   28.66  0.0  0.0   
2066  1.0  17540.70   36.20  62.786667  22.17  0.57   13.45  0.0  0.0   
2067  0.0  83891.90   74.40  61.020000  28.92  0.00   45.47  0.0  0.0   
2068  2.0  28220.80   38.95  38.766667  26.49  0.00   12.46  0.0  0.0   
2069  0.0  28589.10  100.28  15.600000  13.19  0.00   87.09  0.0  0.0   

            9   ...   90   91   92   93   94   95   96   97   98   99  
0     0.000000  ...  0.0  0.0  1.0  0.0

In [117]:
#Train-Test Split on Filter feature data
X_train, X_test, y_train, y_test= train_test_split(new_combined_one_hot_F,y_train, test_size=0.2, random_state=1)

In [118]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1656, 100)
(414, 100)
(1656,)
(414,)


In [119]:
#Construct a Random Forest Classifier on text data-Filtering
clf=RandomForestClassifier()
RF_text = clf.fit(X_train,y_train)
print("Accuracy score (training): {0:.6f}".format(clf.score(X_train, y_train)))
rf_predictions = clf.predict(X_train)
print("Confusion Matrix:")
print(confusion_matrix(y_train, rf_predictions))
print("Classification Report")
print(classification_report(y_train, rf_predictions))


print("Accuracy score (test): {0:.6f}".format(clf.score(X_test, y_test)))
rf_predictions = clf.predict(X_test)
print("Confusion Matrix:")
print(confusion_matrix(y_test, rf_predictions))
print("Classification Report")
print(classification_report(y_test, rf_predictions))


Accuracy score (training): 0.969807
Confusion Matrix:
[[626  28]
 [ 22 980]]
Classification Report
              precision    recall  f1-score   support

   Cancelled       0.97      0.96      0.96       654
     Current       0.97      0.98      0.98      1002

    accuracy                           0.97      1656
   macro avg       0.97      0.97      0.97      1656
weighted avg       0.97      0.97      0.97      1656

Accuracy score (test): 0.852657
Confusion Matrix:
[[124  26]
 [ 35 229]]
Classification Report
              precision    recall  f1-score   support

   Cancelled       0.78      0.83      0.80       150
     Current       0.90      0.87      0.88       264

    accuracy                           0.85       414
   macro avg       0.84      0.85      0.84       414
weighted avg       0.86      0.85      0.85       414



In [120]:
#Construct a Decision Tree Classifier on text data-Filtering
dtc=DecisionTreeClassifier()
DT_text = dtc.fit(X_train,y_train)
print("Accuracy score (training): {0:.6f}".format(dtc.score(X_train, y_train)))
dt_predictions = dtc.predict(X_train)
print("Confusion Matrix:")
print(confusion_matrix(y_train, dt_predictions))
print("Classification Report")
print(classification_report(y_train, dt_predictions))

print("Accuracy score (test): {0:.6f}".format(dtc.score(X_test, y_test)))
dt_predictions = dtc.predict(X_test)
print("Confusion Matrix:")
print(confusion_matrix(y_test, dt_predictions))
print("Classification Report")
print(classification_report(y_test, dt_predictions))

Accuracy score (training): 0.969807
Confusion Matrix:
[[639  15]
 [ 35 967]]
Classification Report
              precision    recall  f1-score   support

   Cancelled       0.95      0.98      0.96       654
     Current       0.98      0.97      0.97      1002

    accuracy                           0.97      1656
   macro avg       0.97      0.97      0.97      1656
weighted avg       0.97      0.97      0.97      1656

Accuracy score (test): 0.835749
Confusion Matrix:
[[126  24]
 [ 44 220]]
Classification Report
              precision    recall  f1-score   support

   Cancelled       0.74      0.84      0.79       150
     Current       0.90      0.83      0.87       264

    accuracy                           0.84       414
   macro avg       0.82      0.84      0.83       414
weighted avg       0.84      0.84      0.84       414



In [121]:
#Feature Selection Using Wrapper type-RandomForest

y_train = CustInfoData["TARGET"]

rfc = RandomForestClassifier()
sfs1 = sfs(rfc,k_features=5,forward=True,floating=False,verbose=2, scoring='accuracy',cv=5)
sfs1= sfs1.fit(combined_one_hot,y_train)

#Select features from Random Forest

feat_cols = list(sfs1.k_feature_idx_)
print(feat_cols)
print(sfs1.k_feature_names_)
print(sfs1.k_score_)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 377 out of 377 | elapsed:  5.3min finished

[2020-10-30 01:55:09] Features: 1/5 -- score: 0.8647342995169082[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 376 out of 376 | elapsed:  7.8min finished

[2020-10-30 02:02:58] Features: 2/5 -- score: 0.8763285024154589[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 375 out of 375 | elapsed:  7.4min finished

[2020-10-30 02:10:23] Features: 3/5 -- score: 0.8801932367149758[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 

[2, 3, 5, 22, 363]
('Usage', 'Age', 'LongDistance', 13, 'Sex_F')
0.8884057971014492


[Parallel(n_jobs=1)]: Done 373 out of 373 | elapsed:  7.7min finished

[2020-10-30 02:26:20] Features: 5/5 -- score: 0.8884057971014492

In [122]:
#Cross Validation
y_train = CustInfoData["TARGET"]
rfc_cv_score = cross_val_score(rfc,combined_one_hot,y_train, cv=20,scoring='balanced_accuracy')
print(rfc_cv_score)

[0.84746419 0.82675184 0.813976   0.89566396 0.825      0.8390625
 0.81875    0.821875   0.8171875  0.8765625  0.86369048 0.90119048
 0.95456349 0.82400794 0.85238095 0.81825397 0.91825397 0.92619048
 0.86031746 0.85575397]


In [127]:
#Train-Test Split on Wrapper feature data-Random Forest

sfs1_selected_features= combined_one_hot[['Usage', 'LongDistance',13, 'Sex_F', 'Paymethod_Auto']]
y_train = CustInfoData["TARGET"]
X_train, X_test, y_train, y_test= train_test_split(sfs1_selected_features,y_train, test_size=0.2, random_state=1)

In [128]:
#Decision Tree on Test-Train Split data on Wrapper feature data-Random Forest
dtc1=DecisionTreeClassifier()
DT_text = dtc1.fit(X_train,y_train)
print("Accuracy score (training): {0:.6f}".format(dtc1.score(X_train, y_train)))
dt_predictions = dtc1.predict(X_train)
print("Confusion Matrix:")
print(confusion_matrix(y_train, dt_predictions))
print("Classification Report")
print(classification_report(y_train, dt_predictions))

print("Accuracy score (test): {0:.6f}".format(dtc1.score(X_test, y_test)))
dt_predictions = dtc1.predict(X_test)
print("Confusion Matrix:")
print(confusion_matrix(y_test, dt_predictions))
print("Classification Report")
print(classification_report(y_test, dt_predictions))

Accuracy score (training): 0.924517
Confusion Matrix:
[[600  54]
 [ 71 931]]
Classification Report
              precision    recall  f1-score   support

   Cancelled       0.89      0.92      0.91       654
     Current       0.95      0.93      0.94      1002

    accuracy                           0.92      1656
   macro avg       0.92      0.92      0.92      1656
weighted avg       0.93      0.92      0.92      1656

Accuracy score (test): 0.862319
Confusion Matrix:
[[132  18]
 [ 39 225]]
Classification Report
              precision    recall  f1-score   support

   Cancelled       0.77      0.88      0.82       150
     Current       0.93      0.85      0.89       264

    accuracy                           0.86       414
   macro avg       0.85      0.87      0.86       414
weighted avg       0.87      0.86      0.86       414



In [129]:
#Feature Selection Using Wrapper type-Decision Tree

y_train = CustInfoData["TARGET"]
dt = DecisionTreeClassifier()
sfs2 = sfs(dt,k_features=5,forward=True,floating=False,verbose=2, scoring='accuracy',cv=5)
sfs2= sfs2.fit(combined_one_hot,y_train)
feat_cols2 = list(sfs2.k_feature_idx_)
print(feat_cols2)
print(sfs2.k_feature_names_)
print(sfs2.k_score_)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 377 out of 377 | elapsed:    7.4s finished

[2020-10-30 02:28:46] Features: 1/5 -- score: 0.8657004830917874[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 376 out of 376 | elapsed:    9.1s finished

[2020-10-30 02:28:55] Features: 2/5 -- score: 0.8734299516908213[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 375 out of 375 | elapsed:    9.2s finished

[2020-10-30 02:29:04] Features: 3/5 -- score: 0.8782608695652174[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 

[2, 5, 20, 113, 363]
('Usage', 'LongDistance', 11, 104, 'Sex_F')
0.881159420289855


[Parallel(n_jobs=1)]: Done 373 out of 373 | elapsed:    9.8s finished

[2020-10-30 02:29:25] Features: 5/5 -- score: 0.881159420289855

In [130]:
#Cross Validation
y_train = CustInfoData["TARGET"]
dt_cv_score = cross_val_score(dt,combined_one_hot,y_train, cv=20,scoring='balanced_accuracy')
print(dt_cv_score)

[0.78339141 0.81087882 0.89508324 0.86759582 0.825      0.865625
 0.75625    0.7828125  0.8109375  0.83125    0.83650794 0.86825397
 0.89325397 0.71170635 0.87281746 0.81488095 0.84107143 0.88531746
 0.80357143 0.86150794]


In [131]:
#Train-Test Split on Wrapper feature data-Decision Tree

sfs2_selected_features= combined_one_hot[['Usage', 'LongDistance', 11,104, 'Sex_F']]
y_train = CustInfoData["TARGET"]
X_train, X_test, y_train, y_test= train_test_split(sfs2_selected_features,y_train, test_size=0.2, random_state=1)

In [132]:
#Random Forest on Test-Train Split data on Wrapper feature data-Decision Tree
rfcc=RandomForestClassifier()
rfcc_text = rfcc.fit(X_train,y_train)
print("Accuracy score (training): {0:.6f}".format(rfcc.score(X_train, y_train)))
rfcc_predictions = rfcc.predict(X_train)
print("Confusion Matrix:")
print(confusion_matrix(y_train, rfcc_predictions))
print("Classification Report")
print(classification_report(y_train, rfcc_predictions))

print("Accuracy score (test): {0:.6f}".format(rfcc.score(X_test, y_test)))
rfcc_predictions = rfcc.predict(X_test)
print("Confusion Matrix:")
print(confusion_matrix(y_test, rfcc_predictions))
print("Classification Report")
print(classification_report(y_test, rfcc_predictions))

Accuracy score (training): 0.922101
Confusion Matrix:
[[591  63]
 [ 66 936]]
Classification Report
              precision    recall  f1-score   support

   Cancelled       0.90      0.90      0.90       654
     Current       0.94      0.93      0.94      1002

    accuracy                           0.92      1656
   macro avg       0.92      0.92      0.92      1656
weighted avg       0.92      0.92      0.92      1656

Accuracy score (test): 0.857488
Confusion Matrix:
[[126  24]
 [ 35 229]]
Classification Report
              precision    recall  f1-score   support

   Cancelled       0.78      0.84      0.81       150
     Current       0.91      0.87      0.89       264

    accuracy                           0.86       414
   macro avg       0.84      0.85      0.85       414
weighted avg       0.86      0.86      0.86       414

